In [58]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [59]:
data=pd.read_csv('ACME-HappinessSurvey2020.csv')
#print(data)
data1=data.drop(columns=['X2','X4'])
#print(data1)

In [60]:

#train = data[data.train==True]
X = data1.drop(columns=['Y'])
#ss = StandardScaler()
#X = ss.fit_transform(X) 
Y = data1.Y

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.15, random_state=0) 
#data is small, dont waste data in testing, test size 0.15
#random state for small data might play a role

In [62]:
print(X.shape, Y.shape)


(126, 4) (126,)


In [63]:
print(Y)

0      0
1      0
2      1
3      0
4      0
      ..
121    1
122    1
123    1
124    0
125    0
Name: Y, Length: 126, dtype: int64


In [64]:
from sklearn.neighbors import KNeighborsClassifier
import time, math
from sklearn.metrics import accuracy_score
cols_results=['family','model','classification_rate','runtime']
results = pd.DataFrame(columns=cols_results)
kVals = range(1,10)
knn_names = ['KNN-'+str(k) for k in kVals]
for k in kVals:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    time_start = time.time()
    y_pred = knn.predict(X_test)
    time_run = time.time()-time_start
    
    results = results.append(pd.DataFrame([['KNN',knn_names[k-1],accuracy_score(y_test,y_pred),time_run]],columns=cols_results),ignore_index=True)
results[results.family=='KNN']



,family,model,classification_rate,runtime
0,KNN,KNN-1,0.578947,0.001991
1,KNN,KNN-2,0.526316,0.002000
2,KNN,KNN-3,0.473684,0.002000
3,KNN,KNN-4,0.526316,0.000999
4,KNN,KNN-5,0.526316,0.000999
5,KNN,KNN-6,0.578947,0.001000
6,KNN,KNN-7,0.578947,0.000940
7,KNN,KNN-8,0.526316,0.000988
8,KNN,KNN-9,0.578947,0.001000


In [65]:
#feature engineering, check if all features are required, correlation matrix, see if any features are not correlated with target variable, then drop
#use hyper parameter tuning, grid search cv , not manual selecting 
#suggest logistic regression decision trees , naive bayes , random forest, support vector machines (SVM) --> grid search on all these, dont default the same 
#from one of these methods you will get 73% accuracy 

In [66]:
  data1.corr().round(3) ## look for closest to 0
# try just dropping X2, X4
## feature selection
#hyper parametric tuning -- what are parameters, hyperparameters

,Y,X1,X3,X5,X6
Y,1.000,0.280,0.151,0.225,0.168
X1,0.280,1.000,0.283,0.433,0.412
X3,0.151,0.283,1.000,0.358,0.204
X5,0.225,0.433,0.358,1.000,0.320
X6,0.168,0.412,0.204,0.320,1.000


In [67]:
### NAIVE BAYES ###

from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X_train, y_train)
predicted=model.predict(X_test)

from sklearn.metrics import (
accuracy_score,
confusion_matrix,
ConfusionMatrixDisplay,
f1_score)

y_pred=model.predict(X_test)
accuracy=accuracy_score(y_pred,y_test)
f1=f1_score(y_pred, y_test, average='weighted')
print('accuracy:', accuracy)
print('f1 score:', f1)

accuracy: 0.6842105263157895
f1 score: 0.6842105263157895


In [69]:
#### GRID SEARCH CV ###
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
model=SVC()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
grid_search = GridSearchCV(SVC(), 
                           param_grid=param_grid) 
grid_search.fit(X_train, y_train)
#print(classification_report(y_test,predictions))

ValueError: Invalid parameter max_depth for estimator SVC(). Check the list of available parameters with `estimator.get_params().keys()`.

In [41]:
### decision tree ####
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
clf=DecisionTreeClassifier()
clf=clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("ACCURACY:",metrics.accuracy_score(y_test,y_pred))

#how to determine different grid --> look up for decision trees

ACCURACY: 0.5789473684210527


In [37]:
## SVM ###
svm_predict=SVC(kernel='linear')
svm_predict.fit(X_train,y_train)
y_pred_SVM=clf.predict(X_test)

print("ACCURACY:",metrics.accuracy_score(y_test,y_pred_SVM))

#grid_search = GridSearchCV(SVC(),
#grid is different for every model 

ACCURACY: 0.5789473684210527


In [38]:

rf=RandomForestClassifier()


In [39]:

param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [3, 6, 9], 
    'max_leaf_nodes': [3, 6, 9], 
} 

grid_search = GridSearchCV(RandomForestClassifier(), 
                           param_grid=param_grid) 
grid_search.fit(X_train, y_train) 
print(grid_search.best_estimator_) 

RandomForestClassifier(max_depth=3, max_features=None, max_leaf_nodes=9,
                       n_estimators=25)


In [40]:

model_grid = RandomForestClassifier(max_depth=3, 
                                    max_features=None, 
                                    max_leaf_nodes=9, 
                                    n_estimators=25) 
model_grid.fit(X_train, y_train) 
y_pred_grid = model.predict(X_test) 
print(classification_report(y_pred_grid, y_test)) 

              precision    recall  f1-score   support

           0       0.33      0.43      0.38         7
           1       0.60      0.50      0.55        12

    accuracy                           0.47        19
   macro avg       0.47      0.46      0.46        19
weighted avg       0.50      0.47      0.48        19



In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
random_search = RandomizedSearchCV(RandomForestClassifier(), 
                                   param_grid) 
random_search.fit(X_train, y_train) 
print(random_search.best_estimator_)

model_random = RandomForestClassifier(max_depth=3, 
                                      max_features='log2', 
                                      max_leaf_nodes=6, 
                                      n_estimators=100) 
model_random.fit(X_train, y_train) 
y_pred_rand = model.predict(X_test) 
print(classification_report(y_pred_rand, y_test)) 